library

In [1]:
import json
import string
import random
from typing import List, Dict
from spacy import displacy
import copy

define function and class

In [2]:
class Dataset:
    data = None
    def __init__(self,data=None):
        self.data = data if data is not None else dict()
        self.fields = list(data.keys())
        
    def __getitem__(self, index):
        assert isinstance(index, (int,str))
        if isinstance(index, int):
            return {k:self.data[k][index] for k in self.data}
        if isinstance(index, str):
            return self.data[index]
        
    def __iter__(self):
        length = len(self.data[self.fields[0]])
        for i in range(length):
            yield self[i]

            
def get_json():
    file_name = '../dataset/bkk-budget-ner-preds.json'
    dataset = None
    with open(file_name) as fp:
        dataset = json.load(fp)
    return dataset


def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))


def to_ner_label_format(segment):
    return {
        "id": id_generator(),
        "from_name": "label",
        "to_name": "text",
        "type": "labels",
        "value": {
            "start": segment['start'],
            "end": segment["end"],
            "score": segment["score"],
            "text": segment["word"],
            "labels": [segment["entity_group"]]
    }}


def get_ls_format(data):
    text = data['text']#.replace('[END_LINE]','\n').replace('[BEGIN_DOC]','\n')
    text, entity = remove_special_tokens(text,data['prediction'])
    results = [to_ner_label_format(x) for x in entity if x['score'] > 0.85]
    return {
    "data": {"text": text},
    "predictions" : [{
        "model_version": "0.3",
        "result": results
    }]
}

In [3]:
def hf_to_spacy_format(entity):
    return {'start': entity['start'],
            'end': entity['end'],
            'label': entity["entity_group"]}


def remove_special_tokens(text: str, tag_list: List[Dict]):
    special_tokens = [{'token':'[END_LINE]',  'rep': '\n'},
                      {'token':'[BEGIN_DOC]', 'rep':''},
                      {'token':'[BEG_LINE]', 'rep': ''}]
    
    for special_token in special_tokens:
        while text.find(special_token['token']) != -1:
            tok_str = special_token['token']
            tok_rep = special_token['rep']

            # get first index of token
            start_index = text.index(tok_str)
            
            # repalce frist special token
            # with replacing token
            text = text.replace(tok_str, tok_rep, 1)
            for tag in tag_list:
                # if the tag start after end_line token
                # then it should subtract by length of the token
                # and add with length of replacing token
                if tag['start'] > start_index:
                    tag['start'] -= len(special_token['token']) - len(tok_rep)
                    tag['end'] -= len(special_token['token']) - len(tok_rep)
                # else if the tag is end after start of token then
                elif tag['end'] > start_index:
                    # if end is replacing token then move end to the left
                    add_one = tag['end'] - len(tok_str) == start_index
                    tag['end'] -= len(tok_str) - (0 if add_one else len(tok_rep))

    return text, tag_list

load dataset

In [4]:
ds = Dataset(get_json())

see example

In [41]:
def visualize_token(text, tags):
    text, cleaned_tags = remove_special_tokens(
        text,
        [hf_to_spacy_format(tag.copy())
         for tag in tags
         if tag['score'] > .85])

    displacy.render({'text': text, 'ents': cleaned_tags}, style="ent", manual=True)

In [5]:
seq_index = 16029
text = ds[seq_index]['text']
tags = ds[seq_index]['prediction']

text, cleaned_tags = remove_special_tokens(
    text,
    [hf_to_spacy_format(tag.copy())
     for tag in tags
     if tag['score'] > .85]
)

displacy.render({'text': text, 'ents': cleaned_tags}, style="ent", manual=True)

## Reformat
reformat dataset to fit into label studio format

In [6]:
# formated_ds = [get_ls_format(copy.deepcopy(x)) for x in ds]
# len(formated_ds)a

In [7]:
# with open('LS-bkk-ner-preds.json', 'w') as fp:
#     json.dump(formated_ds, fp, ensure_ascii=False, indent=2)

# Contructing

In [8]:
import re

In [9]:
def get_patern_of_bullet(String):
    regx = [
        (r'^\d[\-\d]+$', 70),
        (r'^\(\d*(\.?\d*)*\)$', 50),
        (r'^[1-9]\d*(\.[1-9]\d*)*\)$', 20),
        (r'^[1-9]\d*(\.[1-9]\d*)+$', 5),
        (r'^[1-9]\d*\.$', 4),
        (r'^โครงการ', 3),
        (r'^งาน', 3),
        (r'^แผนงาน', 2),
        (r'^[\u0E00-\u0E7F]\.$', 1)
    ]
    
    if String in ['ด้านการจัดบริการของสำนักงานเขต',
                  'ด้านการบริหารจัดการและบริหารราชการกรุงเทพมหานคร',
                  'ด้านการศึกษา',
                  'ด้านความปลอดภัยและความเป็นระเบียบเรียบร้อย',
                  'ด้านทรัพยากรธรรมชาติและสิ่งแวดล้อม',
                  'ด้านพัฒนาสังคมและชุมชนเมือง',
                  'ด้านการสาธารณสุข',
                  'ด้านสาธารณสุข',
                  'ด้านเมืองและการพัฒนาเมือง',
                  'ด้านการระบายนำและบำบัดนำเสีย',
                  'ด้านการบริหารทั่วไป',
                  'ด้านบริหารทั่วไป',
                  'ด้านการโยธาและระบบจราจร',
                  'ด้านเศรษฐกิจและการพาณิชย์']: return ('ด้าน',1)

    for r, l in regx:
        if re.match(r, String):
            if l in [5, 20, 50]:
                l = String.count('.') + l
            if r == '^งาน' and String == 'งานที่จะทำ': continue
            return r, l
    return '', 0

In [10]:
import pandas
import numpy

In [11]:
ds = Dataset(get_json())
toc = pandas.read_csv('../table-of-content.csv',index_col=0)
toc = toc.set_index('path')
toc.loc[toc.start.isna().index, 'start'] = 1

In [12]:
# content = pandas.DataFrame(list(ds))
# is_interested_docs = content.pdf_name.apply(lambda pdf: pdf[4:] in toc.index)
# is_terested_page   = content[is_interested_docs][['pdf_name','page']]\
#     .apply(lambda row: row.page >= toc.loc[row.pdf_name[4:]].start - 1,
#            axis=1)
# fdf = content[is_interested_docs][is_terested_page]

In [13]:
content = pandas.DataFrame(list(ds))
is_interested_docs = content.pdf_name.apply(lambda pdf: pdf[4:] in toc.index and pdf[4:6] != '65')
is_terested_page   = content[is_interested_docs][['pdf_name','page']]\
    .apply(lambda row: (row.page < toc.loc[row.pdf_name[4:]].end - 1) and
            (row.page >= toc.loc[row.pdf_name[4:]].start - 1),
           axis=1)
fdf = content[is_interested_docs][is_terested_page]

In [14]:
def is_not_overlab_entry(entry, entries: List):
    for other in entries:
        if other is entry: continue
        elif other['start'] <= entry['start'] and\
             entry['end'] <= other['end']:
            return False
    return True

In [15]:
def get_line_spans(text:str):
    segments = []
    start_line_idx = 0
    while text.find('\n') != -1:
        lf_idx = text.index('\n')
        segments.append((start_line_idx,lf_idx))
        start_line_idx = lf_idx + 1
        text = text.replace('\n','X',1)
    return segments

def flexing_entry(entries: List, line_spans):
    for entry in entries:
        e_start = entry['start']
        e_end = entry['end']
        for span in line_spans:
            if e_start > span[0] and e_start < span[1]:
                entry['start'] = span[0]
            if e_end > span[0] and e_end < span[1]:
                entry['end'] = span[1]
    

def get_entry_string(text, entry_spans):
    text, entries = remove_special_tokens(
        text,
        [ent for ent in copy.deepcopy(entry_spans) if ent['score'] > 0.85],
    )

    line_spans = get_line_spans(text)
    # if the entry doesn't start at the beginning of
    # line, then flexing it
    flexing_entry(entries, line_spans)

    # getrid of everlaping entry
    non_overlap_entries = [ent for ent in entries
                           if is_not_overlab_entry(ent, entries)]
    return text, non_overlap_entries

In [16]:
# anytree
from anytree import NodeMixin, RenderTree
from anytree import PostOrderIter, AsciiStyle

In [17]:
class Bullet:

    def __init__(self, level, group, text):
        self.level = level
        self.group = group
        self.text = text

    def __eq__(self, value):
        if isinstance(value, str):
            return self.group == value
        return self.level == self._to_generic(value)

    def __le__(self, value):
        return self.level <= self._to_generic(value)

    def __lt__(self, value):
        return self.level  < self._to_generic(value)

    def _to_generic(self, other):
        if isinstance(other, (Bullet,)):
            return other.level
        elif isinstance(other, (int, float)):
            return other
        raise TypeError()

class BudgetEntry(NodeMixin):

    def __init__(self, text, bullet: Bullet=None, parent=None, children=None):
        super(BudgetEntry, self).__init__()
        self.text = text
        self.bullet = bullet if bullet else 0
        self.parent = parent
        try:
            self.amount = re.findall(r'([\d,]+) ?บาท', self.text)[-1]
            self.text = re.sub(r'([\d,]+) ?บาท', '', self.text)
        except IndexError:
            self.amount = None
        if children: self.children = children
            
    def code(self,):
        return ' '.join(re.findall(r'^0[\d]+', self.text))
    
    def name(self,):
        return re.sub(r'^[\d\-\.ก-ส ]+ ([\u0E00-\u0E7F]{2,})', r'\1', self.text)

    def __repr__(self,):
        return "{}".format(self.text.replace('\n', ' '))

In [18]:
def plant_tree(text_entries: List[str], root_name):
    """🌲🌳🎄
    """
    entry_list: List[BudgetEntry] = [None]
    root = BudgetEntry(root_name, -1)
    curr = root
    for entry in text_entries:
        bullet = Bullet(0,'', '')
        for s in entry.split()[:3]:
            group, level = get_patern_of_bullet(s)
            b = Bullet(level, group, s)
            if not level: continue
            bullet = min(bullet, b) if bullet.level else b
        if bullet == 0: continue
        while curr.bullet != -1 and bullet <= curr.bullet:
            curr = curr.parent
        curr = BudgetEntry(entry, bullet, parent=curr)
    return root

In [19]:
all_roots = [BudgetEntry(str(y), -2) for y in toc.fisical_year.unique()]
for pdf_name, pdf_df in fdf.groupby('pdf_name'):
    text_entries = []
    for i, row in pdf_df.iterrows():
        text, entries = get_entry_string(row.text, row.prediction)
#         displacy.render({'text': text, 'ents': [hf_to_spacy_format(e) for e in entries]},style="ent", manual=True)
        text_entries += [text[l['start']:l['end']] for l in entries]
    root = plant_tree(text_entries, pdf_name)
    for year_node in all_roots:
        if year_node.text == pdf_df.iloc[0].pdf_name[4:6]:
            root.parent = year_node
    if root.parent is None: raise ValueError()

In [26]:
rows = []
for year_root in all_roots:
    for node_budget in PostOrderIter(year_root):
        if node_budget.bullet != '^โครงการ': continue
        template = {
            "fisical_year": None,
            "name_organization": None,
            "pdf_name": None, 
            "pdf_url": None,
            **{f"{rtype}_{i+1}": None for i in range(4) for rtype in ['bullet', 'name', 'amount']},
        }
        ancestor_id = 1
        for node in node_budget.ancestors:
            if node.bullet == -2:
                template['fisical_year'] = node.text
            elif node.bullet == -1:
                template['name_organization'] = toc.loc[node.text[4:]]['name']
                template['pdf_name'] = node.text[4:]
                template['pdf_url']  = toc.loc[node.text[4:]].url
            else:
                template[f'bullet_{ancestor_id}'] = node.code()
                template[f'name_{ancestor_id}'] = node.name()
                template[f'amount_{ancestor_id}'] = node.amount
                ancestor_id += 1
        template['output/proj'] = node_budget.code()
        template['output_proj_name'] = node_budget.name()
        template['amount'] = node_budget.amount
        template['node'] = node_budget
        rows.append(template)

In [27]:
bkk_bud_df=pandas.DataFrame(rows).dropna(how='all')

In [34]:
bkk_bud_df.loc[52].node

โครงการก่อสร้างถนนศรีนครินทร์-ร่มเกล้า ช่วงที่ 3 

In [ ]:
bkk_bud_df.to_csv('../bkk-budget-2018-2021.csv',index=0)

In [38]:
print(RenderTree(bkk_bud_df.loc[52].node.parent.parent.parent))

pdf/61/A20171003162738.pdf
└── 03 ด้านการโยธาและระบบจราจร 
    ├── 0301 1. แผนงานบริหารทั่วไป 
    │   ├── 0301001 1.1 งานบริหารทั่วไปด้านการโยธาและระบบจราจร 
    │   ├── 0301001-53-04 1.2 โครงการติดตั้งระบบความปลอดภัยและระบบ ควบคุมอาคารอัตโนมัติ อาคารศาลาว่าการ กรุงเทพมหานคร 2 ดินแดง (ระยะที่ 2) 
    │   ├── 0301001-53-06 1.3 โครงการติดตั้งระบบความปลอดภัยและระบบ ควบคุมอาคารอัตโนมัติ ศาลาว่าการ กรุงเทพมหานคร 2 ดินแดง (ระยะที่ 3) 
    │   ├── 0301001-54-02 1.4 โครงการปรับปรุงอาคารสำนักการโยธา และอาคารสำนักการระบายน้ำ 
    │   ├── 0301001-55-01 1.5 โครงการก่อสร้างระบบควบคุมการจราจรและ ความปลอดภัย ศาลาว่าการกรุงเทพมหานคร 2 ดินแดง (ระยะที่ 1) 
    │   ├── 0301001-55-06 1.6 โครงการก่อสร้างตกแต่งภายในอาคารหอประชุม สภากรุงเทพมหานคร ศาลาว่าการ กรุงเทพมหานคร 2 ดินแดง 
    │   ├── 0301034 1.7 งานแผนงานและประสานสาธารณูปโภค 
    │   └── 0301034-60-01 1.8 โครงการพัฒนาระบบบริหารงานโครงสร้างพื้นฐาน แบบบูรณาการของสำนักการโยธา กรุงเทพมหานคร 
    └── 0310 2. แผนงานการโยธา 
        ├── 0310039 2.1 งานสำร